In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots
import plotly.io as pio
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
from fpdf import FPDF
import os

In [2]:
victims = pd.read_csv('C:\\Users\\ARUN\\Downloads\\archive (6)\\20_Victims_of_rape.csv')
police_hr = pd.read_csv('C:\\Users\\ARUN\\Downloads\\archive (6)\\35_Human_rights_violation_by_police.csv')
auto_theft = pd.read_csv('C:\\Users\\ARUN\\Downloads\\archive (6)\\30_Auto_theft.csv')
prop_theft = pd.read_csv('C:\\Users\\ARUN\\Downloads\\archive (6)\\10_Property_stolen_and_recovered.csv')

In [3]:
victims

,Area_Name,Year,Subgroup,Rape_Cases_Reported,Victims_Above_50_Yrs,Victims_Between_10-14_Yrs,Victims_Between_14-18_Yrs,Victims_Between_18-30_Yrs,Victims_Between_30-50_Yrs,Victims_of_Rape_Total,Victims_Upto_10_Yrs
0,Andaman & Nicobar Islands,2001,Total Rape Victims,3,0,0,3,0,0,3,0
1,Andaman & Nicobar Islands,2001,Victims of Incest Rape,1,0,0,1,0,0,1,0
2,Andaman & Nicobar Islands,2001,Victims of Other Rape,2,0,0,2,0,0,2,0
3,Andaman & Nicobar Islands,2002,Total Rape Victims,2,0,0,1,1,0,2,0
4,Andaman & Nicobar Islands,2002,Victims of Incest Rape,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1045,West Bengal,2009,Victims of Incest Rape,3,0,0,1,1,1,3,0
1046,West Bengal,2009,Victims of Other Rape,2333,0,18,84,1834,391,2333,6
1047,West Bengal,2010,Total Rape Victims,2311,0,22,51,1779,459,2311,0
1048,West Bengal,2010,Victims of Incest Rape,4,0,1,0,2,1,4,0


In [4]:
ly=victims['Year'].max()
ly=str(ly)
ly

'2010'

In [5]:
pdf = FPDF()

pdf.add_page()
pdf.set_font("Arial", style="B", size=13)
 
pdf.text(10,10,"Rape case")
pdf.ln()
pdf.set_font("Arial",size=12)
pdf.text(50,10,"Incest rape cases reported(2001 -"+ly+")")



In [6]:
inc_victims = victims[victims['Subgroup']=='Victims of Incest Rape']

g = pd.DataFrame(inc_victims.groupby(['Year'])['Rape_Cases_Reported'].sum().reset_index())
g.columns = ['Year','Cases Reported']
image_file = 'temp.png'
fig = px.bar(g,x='Year',y='Cases Reported',color_discrete_sequence=['blue'])
image_bytes = pio.to_image(fig, format='png')
with open(image_file, 'wb') as f:
    f.write(image_bytes)
pdf.image(image_file, x=30, y=30, w=190)


In [7]:
pdf.add_page()
pdf.set_font("Arial",size=12)
pdf.text(50,10,"State/UT wise incest rape cases (2001 -"+ly+")")
pdf.ln()
g1 = pd.DataFrame(inc_victims.groupby(['Area_Name'])['Rape_Cases_Reported'].sum().reset_index())
g1.columns = ['State/UT','Cases Reported']
g1.replace(to_replace='Arunachal Pradesh',value='Arunanchal Pradesh',inplace=True)

shp_gdf = gpd.read_file('C:\\Users\\ARUN\\Downloads\\archive (7)\\India States\\Indian_states.shp')
merged = shp_gdf.set_index('st_nm').join(g1.set_index('State/UT'))

fig, ax = plt.subplots(1, figsize=(10, 10))
ax.axis('off')
ax.set_title('State-wise Rape-Cases Reported (2001-'+ly+")",
             fontdict={'fontsize': '15', 'fontweight' : '3'})
fig = merged.plot(column='Cases Reported', cmap='Reds', linewidth=0.5, ax=ax, edgecolor='0.2',legend=True)
image_file1 = 'temp1.png'
plt.savefig(image_file1, bbox_inches='tight', dpi=300)
plt.close()

    # Add the image to the PDF
pdf.image(image_file1, x=20, y=20, w=190)

    # Delete the temporary image file
os.remove(image_file1)

# Save the PDF file



In [8]:
pdf.add_page()
pdf.set_font("Arial",size=12)
pdf.text(50,10,"Distribution of age groups of victims(2001 -"+ly+")")
pdf.ln()

above_50 = inc_victims['Victims_Above_50_Yrs'].sum()
ten_to_14 = inc_victims['Victims_Between_10-14_Yrs'].sum()
fourteen_to_18 = inc_victims['Victims_Between_14-18_Yrs'].sum()
eighteen_to_30 = inc_victims['Victims_Between_18-30_Yrs'].sum()
thirty_to_50 = inc_victims['Victims_Between_30-50_Yrs'].sum()
upto_10 = inc_victims['Victims_Upto_10_Yrs'].sum()

age_grp = ['Upto 10','10 to 14','14 to 18','18 to 30','30 to 50','Above 50']
age_group_vals = [upto_10,ten_to_14,fourteen_to_18,eighteen_to_30,thirty_to_50,above_50]

fig = go.Figure(data=[go.Pie(labels=age_grp, values=age_group_vals,sort=False,
                            marker=dict(colors=px.colors.qualitative.G10),textfont_size=12)])
pio.write_image(fig, 'output.png')
image_width = 200  # Replace with the desired width of the image
image_height = 200  # Replace with the desired height of the image

# Calculate the position to center the image horizontally
pdf_width = 210  # Width of the PDF page
x = (pdf_width - image_width) / 2

# Calculate the position to center the image vertically
pdf_height = 297  # Height of the PDF page
y = (pdf_height - image_height) / 2

# Add the image
pdf.image('output.png', x=x, y=y, w=image_width, h=image_height)


In [9]:
#pdf=FPDF()
pdf.add_page()
pdf.set_font("Arial", style="B", size=13)
 
pdf.text(10,10,"Human Rights violation by the Police")

pdf.set_font("Arial",size=12)
pdf.text(40,20,'State/UT wise cases registered under Human Rights violation(2001 -'+ly+')')
g2 = pd.DataFrame(police_hr.groupby(['Area_Name'])['Cases_Registered_under_Human_Rights_Violations'].sum().reset_index())
g2.columns = ['State/UT','Cases Reported']

g2.replace(to_replace='Arunachal Pradesh',value='Arunanchal Pradesh',inplace=True)

shp_gdf = gpd.read_file('C:\\Users\\ARUN\\Downloads\\archive (7)\\India States\\Indian_states.shp')
merged = shp_gdf.set_index('st_nm').join(g2.set_index('State/UT'))

fig, ax = plt.subplots(1, figsize=(10, 10))
ax.axis('off')
#ax.set_title('State/UT wise cases registered under Human Rights violation(2001 -'+ly+')',fontdict={'fontsize': '15', 'fontweight' : '3'})
fig = merged.plot(column='Cases Reported', cmap='Oranges', linewidth=0.5, ax=ax, edgecolor='0.2',legend=True)
plt.savefig("out1.png", bbox_inches='tight', dpi=300)
plt.close()

    # Add the image to the PDF
pdf.image("out1.png", x=10, y=20, w=190)


In [10]:
#pdf=FPDF()
pdf.add_page()
pdf.set_font("Arial",size=12)
pdf.text(40,20,'Cases registered against Police under Human Rights violations from (2001 -'+ly+')')
g3 = pd.DataFrame(police_hr.groupby(['Year'])['Cases_Registered_under_Human_Rights_Violations'].sum().reset_index())
g3.columns = ['Year','Cases Registered']

fig = px.bar(g3,x='Year',y='Cases Registered',color_discrete_sequence=['green'])
pio.write_image(fig, 'output.png')
pdf.image("output.png", x=10, y=20, w=190)


In [11]:
#pdf=FPDF()
pdf.add_page()
pdf.set_font("Arial", style="B", size=13)
pdf.cell(0, 10, "Types of Human Rights violations", ln=True, align="C")
x=police_hr.Group_Name.value_counts()

pdf.set_font("Arial", size=12)


pdf.set_fill_color(0, 0, 255)
pdf.set_text_color(255, 255, 255)
pdf.set_draw_color(0, 0, 0)
pdf.set_line_width(0.3)

pdf.cell(120, 10, "Group_Name", 1, 0, "C", True)
pdf.cell(40, 10, "Count", 1, 1, "C", True)

pdf.set_fill_color(255, 255, 255)
pdf.set_text_color(0, 0, 0)
pdf.set_draw_color(0, 0, 0)
pdf.set_line_width(0.2)

for group_name, count in x.items():
    pdf.cell(120, 10, group_name, 1, 0, "L", False)
    pdf.cell(40, 10, str(count), 1, 1, "R", False)




In [12]:

pdf.set_font("Arial", style="B", size=13)
pdf.cell(10, 10, "Cases Registed under Human Rights Violation - Fake encounter killings", "C", True)
fake_enc_df = police_hr[police_hr['Group_Name']=='HR_Fake encounter killings'] 
m=fake_enc_df.Cases_Registered_under_Human_Rights_Violations.sum()
m=str(m)
pdf.set_font("Arial",size=12)
pdf.cell(10, 10,m,"C", True)


In [13]:
pdf.set_font("Arial", style="B", size=13)
pdf.cell(10, 10, "Cases Registed under Human Rights Violation - False implication", "C", True)
false_imp_df = police_hr[police_hr['Group_Name']=='HR_False implication'] 
n=false_imp_df.Cases_Registered_under_Human_Rights_Violations.sum()
n=str(n)
pdf.set_font("Arial",size=12)
pdf.cell(10, 10,n,"C", True)


In [14]:
#pdf=FPDF()
pdf.add_page()
pdf.set_font("Arial", style="B", size=13)
pdf.cell(10, 10, "Policemen Chargesheeted vs Policemen Convicted", "C", True)
g4 = pd.DataFrame(police_hr.groupby(['Year'])['Policemen_Chargesheeted','Policemen_Convicted'].sum().reset_index())

year=['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010']

fig = go.Figure(data=[
    go.Bar(name='Policemen Chargesheeted', x=year, y=g4['Policemen_Chargesheeted'],
           marker_color='purple'),
    go.Bar(name='Policemen Convicted', x=year, y=g4['Policemen_Convicted'],
          marker_color='red')
])

fig.update_layout(barmode='group',xaxis_title='Year',yaxis_title='Number of policemen')
pio.write_image(fig, 'output.png')
pdf.image("output.png", x=10, y=20, w=190)


In [15]:
#pdf=FPDF()
pdf.add_page()
pdf.set_font("Arial", style="B", size=13)
pdf.cell(10, 10, "Auto Theft cases", "C", True)
pdf.set_font("Arial",size=12)
pdf.cell(10,5,"State/UT wise number of vehicles stolen from(2001 -"+ly+")","C", True)
g5 = pd.DataFrame(auto_theft.groupby(['Area_Name'])['Auto_Theft_Stolen'].sum().reset_index())
g5.columns = ['State/UT','Vehicle_Stolen']
g5.replace(to_replace='Arunachal Pradesh',value='Arunanchal Pradesh',inplace=True)

shp_gdf = gpd.read_file('C:\\Users\\ARUN\\Downloads\\archive (7)\\India States\\Indian_states.shp')
merged = shp_gdf.set_index('st_nm').join(g5.set_index('State/UT'))

fig, ax = plt.subplots(1, figsize=(10, 10))
ax.axis('off')
ax.set_title('State-wise Auto Theft Cases Reported(2001 -'+ly+')',
             fontdict={'fontsize': '15', 'fontweight' : '3'})
fig = merged.plot(column='Vehicle_Stolen', cmap='Wistia', linewidth=0.5, ax=ax, edgecolor='0.2',legend=True)
image_file1 = 'temp1.png'
plt.savefig(image_file1, bbox_inches='tight', dpi=300)
plt.close()

    # Add the image to the PDF
pdf.image(image_file1, x=20, y=40, w=190)



In [16]:
#pdf=FPDF()
pdf.add_page()
pdf.set_font("Arial",size=12)
pdf.cell(10,5,"Proportion of vehicles stolen, traced and recovered ","C", True)
auto_theft_traced = auto_theft['Auto_Theft_Coordinated/Traced'].sum()
auto_theft_recovered = auto_theft['Auto_Theft_Recovered'].sum()
auto_theft_stolen = auto_theft['Auto_Theft_Stolen'].sum()

vehicle_group = ['Vehicles Stolen','Vehicles Traced','Vehicles Recovered']
vehicle_vals = [auto_theft_stolen,auto_theft_traced,auto_theft_recovered]

colors = ['crimson','gold','green']

fig = go.Figure(data=[go.Pie(labels=vehicle_group, values=vehicle_vals,sort=False,
                            marker=dict(colors=colors),textfont_size=12)])

pio.write_image(fig, 'output1.png')
pdf.image("output1.png", x=20, y=20, w=190)


In [17]:
#pdf=FPDF()
pdf.add_page()
pdf.set_font("Arial",size=12)
pdf.cell(10,5,"Year wise vehicles stolen","C", True)
g5 = pd.DataFrame(auto_theft.groupby(['Year'])['Auto_Theft_Stolen'].sum().reset_index())

g5.columns = ['Year','Vehicles Stolen']

fig = px.bar(g5,x='Year',y='Vehicles Stolen',color_discrete_sequence=['#17becf'])
pio.write_image(fig, 'output2.png')
pdf.image("output2.png", x=20, y=20, w=190)


In [18]:
#pdf=FPDF()
pdf.add_page()
pdf.set_font("Arial",size=12)
pdf.cell(10,5,"Types of vehicles stolen ","C", True)
vehicle_list = ['Motor Cycles/ Scooters','Motor Car/Taxi/Jeep','Buses',
               'Goods carrying vehicles (Trucks/Tempo etc)','Other Motor vehicles']

sr_no = [1,2,3,4,5]

fig = go.Figure(data=[go.Table(header=dict(values=['Sr No','Vehicle type'],
                                          fill_color='turquoise',
                                           height=30),
                 cells=dict(values=[sr_no,vehicle_list],
                            height=30))
                     ])
pio.write_image(fig, 'output3.png')
pdf.image("output3.png", x=20, y=20, w=190)
#pdf.output("data.pdf")

In [19]:
#pdf=FPDF()
pdf.add_page()
pdf.set_font("Arial",size=12)
pdf.cell(10,5,"Top 10 States/UT with highest number of motor cycles stolen","C", True)
motor_c = auto_theft[auto_theft['Sub_Group_Name']=='1. Motor Cycles/ Scooters']

g8 = pd.DataFrame(motor_c.groupby(['Area_Name'])['Auto_Theft_Stolen'].sum().reset_index())
g8_sorted = g8.sort_values(['Auto_Theft_Stolen'],ascending=True)
fig = px.bar(g8_sorted.iloc[-10:,:], y='Area_Name', x='Auto_Theft_Stolen',
             orientation='h',color_discrete_sequence=['#008080'])

pio.write_image(fig, 'output4.png')
pdf.image("output4.png", x=20, y=20, w=190)
#pdf.output("data.pdf")

In [20]:
#pdf=FPDF()
pdf.add_page()
pdf.set_font("Arial", style="B", size=13)
pdf.cell(10, 10, "Property Theft Case", "C", True)
pdf.set_font("Arial",size=12)
pdf.cell(10,5,"State/UT wise number of property stolen cases from (2001 -"+ly+")","C", True)
g7 = pd.DataFrame(prop_theft.groupby(['Area_Name'])['Cases_Property_Stolen'].sum().reset_index())
g7.columns = ['State/UT','Cases Reported']
g7.replace(to_replace='Arunachal Pradesh',value='Arunanchal Pradesh',inplace=True)

shp_gdf = gpd.read_file('C:\\Users\\ARUN\\Downloads\\archive (7)\\India States\\Indian_states.shp')
merged = shp_gdf.set_index('st_nm').join(g7.set_index('State/UT'))

fig, ax = plt.subplots(1, figsize=(10, 10))
ax.axis('off')
ax.set_title('State-wise Property Stolen-Cases Reported',
             fontdict={'fontsize': '15', 'fontweight' : '3'})
fig = merged.plot(column='Cases Reported', cmap='RdPu', linewidth=0.5, ax=ax, edgecolor='0.2',legend=True)
plt.savefig("out5.png", bbox_inches='tight', dpi=300)
plt.close()

    # Add the image to the PDF
pdf.image("out5.png", x=20, y=30, w=190)
#pdf.output("data.pdf")


In [21]:
#pdf=FPDF()
pdf.add_page()
pdf.set_font("Arial",size=12)
pdf.cell(10,5,"Property stolen vs recovered ","C", True)
prop_theft_recovered = prop_theft['Cases_Property_Recovered'].sum()
prop_theft_stolen = prop_theft['Cases_Property_Stolen'].sum()

prop_group = ['Property Stolen Cases','Property Recovered Cases']
prop_vals = [prop_theft_stolen,prop_theft_recovered]

colors = ['red','green']

fig = go.Figure(data=[go.Pie(labels=prop_group, values=prop_vals,sort=False,
                            marker=dict(colors=colors),textfont_size=12)])
pio.write_image(fig, 'output6.png')
pdf.image("output6.png", x=20, y=20, w=190)
#pdf.output("data.pdf")


In [22]:
#pdf=FPDF()
pdf.add_page()
pdf.set_font("Arial",size=12)
pdf.cell(10,5,"Year-wise Value of Property Stolen and Recovered","C", True)
g9 = pd.DataFrame(prop_theft.groupby(['Year'])['Value_of_Property_Recovered','Value_of_Property_Stolen'].sum().reset_index())

year=['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010']

fig = go.Figure(data=[
    go.Bar(name='Property Recovered', x=year, y=g9['Value_of_Property_Recovered'],
           marker_color='gold'),
    go.Bar(name='Property Stolen', x=year, y=g9['Value_of_Property_Stolen'],
          marker_color='darkblue')
])

fig.update_layout(barmode='group',xaxis_title='Year',yaxis_title='Value in Rupees',
                 title='Year-wise Value of Property Stolen and Recovered')
pio.write_image(fig, 'output7.png')
pdf.image("output7.png", x=20, y=40, w=190)
pdf.output("report.pdf")

''